In [53]:
import requests
import sqlite3
import pandas as pd

def get_data(company, apikey):
  # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={company}&apikey={apikey}&outputsize=full'
  r = requests.get(url)
  data = r.json()
  if 'Error Message' not in data:
    print(data)
  return data

def data_preprocessing(companies, apikey):
  final_data = []
  for company in companies:
    data = get_data(company,apikey)
    if 'Error Message' not in data:
      daily_data_points = data['Time Series (Daily)']
      start_date = '2020-01-01'
      end_date = '2024-05-31'
      for date in daily_data_points:
        # [date , Company, Open, Close, High, Low, Volume]
        open= daily_data_points[date]['1. open']
        high= daily_data_points[date]['2. high']
        low= daily_data_points[date]['3. low']
        close = daily_data_points[date]['4. close']
        volume = daily_data_points[date]['5. volume']
        if date >= start_date and date <= end_date:
          record = [date, company, open, close, high, low, volume]
          final_data.append(record)
  return final_data

In [55]:
# Data Preprocessing
companies = ["LT","INFY", "SBIN", "BHARTIARTL", "ICICIBANK", "HDFCBANK", "RELIANCE", "HINDUNILVR", "ITC","IBM", "TCS"]
apikey = 'SNSJWUIV9ZN42UVQ'
final_data = data_preprocessing(companies, apikey)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'INFY', '3. Last Refreshed': '2024-07-05', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2024-07-05': {'1. open': '19.4500', '2. high': '19.5650', '3. low': '19.3450', '4. close': '19.4300', '5. volume': '7709818'}, '2024-07-03': {'1. open': '19.2900', '2. high': '19.4300', '3. low': '19.2800', '4. close': '19.3700', '5. volume': '7252432'}, '2024-07-02': {'1. open': '19.1000', '2. high': '19.2600', '3. low': '19.0400', '4. close': '19.2300', '5. volume': '10531768'}, '2024-07-01': {'1. open': '18.8800', '2. high': '18.9500', '3. low': '18.7100', '4. close': '18.7600', '5. volume': '8064196'}, '2024-06-28': {'1. open': '18.6900', '2. high': '18.7400', '3. low': '18.5400', '4. close': '18.6200', '5. volume': '9520327'}, '2024-06-27': {'1. open': '18.6200', '2. high': '18.7000', '3. low': '18.5500', '4. close': '18.6300', '5. volume': '8949400'}, '20

In [56]:
columns = ['date', 'company', 'open', 'close', 'high', 'low', 'volume']
final_df = pd.DataFrame(final_data, columns=columns)

In [57]:
#TASK 3
# Create a connection to the SQLite database
conn = sqlite3.connect(f'data.db'
                      #  ,user="username",
                      #  password="password",
                      #  host="host",
                      #  port="port"
                      )
cursor = conn.cursor()

# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS historical_data (
        Date TEXT,
        Company TEXT,
        Open REAL,
        Close REAL,
        High REAL,
        Low REAL,
        Volume INTEGER
    )
''')

# Insert data into the table
final_df.to_sql('historical_data', conn, if_exists='replace', index=False)
conn.commit()

In [59]:
# TASK ============================== 1
def get_historical_data(conn, company):
  query = f'''SELECT *
          FROM
              historical_data
          where Company = '{company}'
          order by Date desc;'''
  results = pd.read_sql_query(query, conn)
  return results
# TASK =============================== 2
def get_yesterday_data(conn, company):
  query = f'''SELECT *
            FROM historical_data
            where Company = '{company}'
            ORDER BY Date DESC
            LIMIT 1 OFFSET 1;'''
  results = pd.read_sql_query(query, conn)
  return results

def run_query(query):
  results = pd.read_sql_query(query, conn)
  return results

In [60]:
res = get_historical_data(conn, "INFY")
print(res)

            date company     open    close     high      low    volume
0     2024-05-31    INFY  16.7300  16.7500  16.8200  16.4350  14253851
1     2024-05-30    INFY  16.9100  16.8700  16.9600  16.7650  11351725
2     2024-05-29    INFY  17.1200  17.0300  17.1450  17.0000   8563881
3     2024-05-28    INFY  17.3100  17.3200  17.3900  17.2700  11508090
4     2024-05-24    INFY  17.4000  17.3600  17.4800  17.3300   9240966
...          ...     ...      ...      ...      ...      ...       ...
1106  2020-01-08    INFY  10.0000  10.0900  10.1200   9.9900  13440377
1107  2020-01-07    INFY  10.1200  10.1000  10.1700  10.1000  10589242
1108  2020-01-06    INFY  10.2200  10.2100  10.2500  10.1700  10068933
1109  2020-01-03    INFY  10.2900  10.3100  10.3500  10.2600   6992166
1110  2020-01-02    INFY  10.3000  10.2900  10.3400  10.2300   5687971

[1111 rows x 7 columns]


In [61]:
result = get_yesterday_data(conn, "INFY")
print(result)

         date company     open    close     high      low    volume
0  2024-05-30    INFY  16.9100  16.8700  16.9600  16.7650  11351725


In [69]:
# TASK ========================= 4
# Indexing for Company Wise Daily Variation of Prices and Company Wise Daily Volume Change
daily_change_indexing = '''CREATE INDEX idx_company_date ON historical_data (Date);'''
query0 = '''SELECT Company, Date, Volume
            FROM historical_data
            ORDER BY Date;'''

# Indexing for Median Daily Variation
median_indexing = '''CREATE INDEX idx_company_date_prices ON historical_data (Company, Date, Close, Open);'''

query = f'''WITH ranked_variations AS (
          SELECT
              Company,
              Date,
              (Close - Open) AS Daily_Variation,
              ROW_NUMBER() OVER (PARTITION BY Company ORDER BY (Close - Open)) AS row_num,
              COUNT(*) OVER (PARTITION BY Company) AS total_count
          FROM historical_data
      )
      SELECT
          Company,
          AVG(Daily_Variation) AS Median_Daily_Variation
      FROM ranked_variations
      WHERE row_num IN (total_count / 2 + 1, (total_count + 1) / 2)
      GROUP BY Company'''

print(run_query(query))

  Company  Median_Daily_Variation
0     IBM                    0.07
1    INFY                    0.01
2     TCS                    0.00
